In [1]:
import cobra
import time
import numpy as np
from cobra import Model, Reaction, Metabolite
import pandas as pd
import more_itertools as mi
import multiprocessing as mp
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
import pickle 

In [4]:
print("LEER MODELO")

LEER MODELO


In [5]:
cobra.Configuration().solver="cplex"

In [6]:
nombre=""

In [7]:
def identificaTipo(nombre):
    if "(" in nombre:
        indice1=nombre.index("(")
        nombre2=nombre[indice1+1:]
        indice2=nombre2.index(")")
        nombre3=nombre2[:indice2] 
        if " and " in nombre3:
            tipo="SOP"
        if " or " in nombre3:
            tipo="POS"
    else:
        tipo="SOP"
    return(tipo)

def toPOS(nombre,tope=3):

    productos=set()    
    tipo=identificaTipo(nombre)

    if tipo=="SOP":
        sumandos=nombre.split(" or ")
        for sumando in sumandos:
            if " and " in sumando:
                casos=sumando.split(" and ")
                miCaso=set()
                for caso in casos:
                    caso2=caso.replace("(","")
                    caso3=caso2.replace(")","")
                    caso4=caso3.replace(" ","")
                    miCaso.add(caso4)

                productos.add(frozenset(miCaso))
            else:
                productos.add(frozenset([sumando]))

        CS=set([frozenset()])
        for producto in productos:
            CS2=set()
            for C in CS:
                if len(set(C).intersection(producto))==0:
                    for a in producto:
                        C2=set(C.copy())
                        C2.add(a)
                        if len(C2)<=tope:
                            CS2.add(frozenset(C2))
                else:
                    CS2.add(C)
            CS=CS2

        CS3=set()
        for C in CS:
            bien=True
            for C2 in CS:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    else:
        CS2=set()
        productos = nombre.split(" and ")
        for producto in productos:
            CS=set()
            componentes=producto.split(" or ")
            for C in componentes:
                C2=C.replace("(","")
                C3=C2.replace(")","")
                C4=C3.replace(" ","")
                CS.add(C4)
            CS2.add(frozenset(CS))
        CS3=set()
        for C in CS2:
            bien=True
            for C2 in CS2:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    return(CS3)

def asociar(modelo,tope=4):
    verbose=False
    total=0
    asociados=dict()
    reacciones=dict()

    for num in range(len(modelo.reactions)):
        nombre=modelo.reactions[num].gene_reaction_rule
        CS3=toPOS(nombre,tope)
        k=0
        while k<=tope:
            mas=0
            nivel=set([C for C in CS3 if len(C)==k and not C==frozenset({''})])
            if len(nivel)>0:
                mios=set()
                for caso in nivel:
                    mio=set()
                    for a in caso:
                        mio.add(a)
                    mios.add(frozenset(mio))

                asociados[(modelo.reactions[num],k)]=mios

                for caso in mios:
                    if not caso in reacciones:
                        reacciones[caso]=set([modelo.reactions[num]])
                    else:
                        reacciones[caso].add(modelo.reactions[num])
                mas+=len(nivel)
            total+=mas
            k+=1    
        if verbose:
            print(num,len(modelo.reactions[num].genes),mas)

    return(asociados,reacciones)
    
    
def soporteG(soporte,tope,modelo):
    sopG=set()
    for j in soporte:
        r=modelo.reactions[j]
        for k in range(1,tope+1):
            if (r,k) in asociados:
                sopG.update(asociados[(r,k)])

    sopG2=set()
    for caso in sopG:
        poner=True
        for caso2 in sopG:
            if not caso==caso2 and caso2.issubset(caso) and len(caso2)>0:
                poner=False
                break
        if poner:
            sopG2.add(caso)
    return(sopG2)

def chequea(inicio,fin):
    respuestas=set()
    for i in range(inicio,fin):
        if i<len(finales):
            C=finales[i]
            vale=True
            if len(C.intersection(CSs1))>0:
                vale = False
            else:
                if C in powers:
                    po=powers[C]
                else:
                    po=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                    powers[C]=po

                if len(po.intersection(todosMCS))>0 or len(evitar.intersection(C))>0:
                    vale=False

                if vale:
                    for sop in misSoportes:
                        if len(po.intersection(sopG[sop]))==0:           
                            vale=False
                            break
            if vale:
                respuestas.add(C)

    return(respuestas)


In [8]:
def analizaSalto(inicio,fin):
    misSoportes2=set()
    
    salto=3
    
    descartados=set()
    miLista2=set()
    
    i2=inicio
    while i2 < fin:
        casos=[]
        contCasos=0
        for k in range(i2,fin):
            if k<len(listaL2) and not listaL2[k] in descartados:
                casos.append(listaL2[k])
                contCasos+=1
            if contCasos>=salto:
                break
        i2=k+1

        cont3=0
        if len(casos) > 0:
            with modelo:
                for caso in casos:
                    descartados.add(caso)
                    for g in caso:
                        modelo.genes.get_by_id(g).knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                
                sol=modelo.slim_optimize(error_value=-1)
                if sol==-1:
                    for caso in casos:
                        miLista2.add(caso)
                else:
                    sol2=modelo.optimize()
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                    misSoportes2.add(sop)
                    nuevos=[caso for caso in sopG[sop] if len(caso)>0]
                    for C in set(listaL2).difference(descartados): 
                        sst=time.time()
                        if C in powers:
                            sub=powers[C]
                        else:
                            sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                            powers[C]=sub
                        if len(sub.intersection(sopG[sop]))==0:
                            descartados.add(C)
                        

    return([miLista2,misSoportes2])

In [9]:
def analiza(inicio,fin):    
    misSoportes2=set()
    descartados=set()
    CSs=set()
    for caso in miLista2[inicio:fin]:
        if not caso in descartados:
            descartados.add(caso)
            with modelo:
                for g in caso:
                    modelo.genes.get_by_id(g).knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                sol=modelo.slim_optimize(error_value=-1)
                if sol==-1:
                    CSs.add(caso)
                else:
                    sol2=modelo.optimize()
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                    misSoportes2.add(sop)
                    nuevos=[caso for caso in sopG[sop] if len(caso)>0]
                    for C in set(miLista2).difference(descartados): 
                        sst=time.time()
                        sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                        if len(sub.intersection(sopG[sop]))==0:
                            descartados.add(C)
                        
    return([CSs,misSoportes2])

In [10]:
def analizaGenes(inicio,fin):
    correctas=set()
    soportes=set()
    for i in range(inicio,fin):
        if i<len(listaGenes):
            g=modelo.genes.get_by_id(listaGenes[i].id)
            with modelo:
                g.knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                sol=modelo.slim_optimize(error_value=-1)
                if sol<0:
                    correctas.add(frozenset([g.id]))
                else:
                    sol2=modelo.optimize(objective_sense="maximize")
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                    soportes.add(sop)
    return([correctas,soportes])

In [ ]:
def CSV2(modelo2,tope,misReacciones,misValores,verbose,evita):
    global tiempos
    global todosMCS
    global evitar
    global misSoportes
    global listaMias
    global listaFinales
    global sopG
    global modelo
    
    global miLista2
    global listaL2
    
    
    global finales
    global CSs1
    
    modelo=modelo2
    
    listaMias=[]
    salto=6
    
    evitar=evita
    listaMCS=[]
    listaMCS.append(set())
    
    listaCandidatos=[]
    listaCandidatos.append(set([frozenset()]))
    todosMCS=set()
    cutsets=[set()]
    candidatos=[set()]

    ts=0
    
    for i in range(1,tope+1):
        if i==1:      
            st=time.time()
            cpus=mp.cpu_count()
            cpus=32
            miL2=set()
            listaGenes=list(modelo.genes)

            k=0
            argus=[]
            num=int(np.floor(len(modelo.genes)/(cpus)))
            if num==0:
                num=1
            while k*num<len(listaGenes):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analizaGenes,argus)

            CS0=set()
            
            for result in results:
                CS0.update(result[0])
                misSoportes.update(result[1])
                
            print(len(CS0),len(misSoportes))
            todosMCS.update(CS0)    
            cutsets.append(CS0)
            
            for sop in misSoportes:
                sopG[sop]=soporteG(sop,tope,modelo)
            
            print("Final Fase 1 con",len(cutsets[1]),"cutsets")

            CSs1=set([g for caso in CS0 for g in caso])
            
            print(time.time()-st)
            print()
       
        ############################## Final Fase 1 ############################################
    
        else:
            print("Inicio de Fase",i,"con",len(misSoportes),"soportes")
            iMio=i
            
            ############################## Parte 1.- Berge #################################### 
            st1=time.time()
            CST2=set()
            lista=set([frozenset()])
            descartados=set()
            cont2=0
            long=dict()
            for sop in misSoportes:
                long[sop]=len(sopG[sop])
            sortedSupports= sorted(long.items(), key=lambda x:x[1])
            finales=set()
            
            for t in range(len(misSoportes)):
                ################ Para cada soporte #################
                item=sortedSupports[t]
                sop=item[0]
                
                lista2=set()
                for caso in lista:
                    sst=time.time()
                    if caso in powers:
                        po=powers[caso]
                    else:
                        po=set([frozenset(a) for a in mi.powerset(caso) if len(a)>0])
                        powers[caso]=po
                    esta=(len(po.intersection(sopG[sop]))>0)
                    ts+=(time.time()-sst)
                    if esta:
                        lista2.add(caso)
                    else:
                        for caso2 in sopG[sop]:
                            if len(caso.union(caso2))<iMio:
                                nuevo=caso.union(caso2)
                                lista2.add(nuevo)
                            if len(caso.union(caso2))==iMio:
                                nuevo=caso.union(caso2)
                                finales.add(nuevo)

                
                lista4=set()
                for C in lista2:
                    sst=time.time()
                    if C in powers:
                        po2=powers[C]
                    else:
                        po2=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                        powers[C]=po2
                    if len(po2.intersection(todosMCS))==0 and len(evitar.intersection(C))==0:
                        lista4.add(C)
                    ts+=(time.time()-sst)
                cont2+=1
                lista=lista4
                lista3=[C for C in lista if len(C)<i]
                if verbose:
                    print("Mal",len(lista3))
            
                
            print("Finales",len(finales))
                    #break
            ########################## He terminado de procesar los soportes ###############
            
            if i==tope:
                print("Fase1",time.time()-st1)
            st2=time.time()
            
            if verbose:
                print("Parte paralela")
            
            lista3=[C for C in lista if len(C)<i]
            
            
            
            misFinales=list(finales)
            if verbose:
                print("Total",len(misFinales))
            lista=set()
            numLista=4*10**6
            r=0
            while r*numLista<len(misFinales):
                maximo=(r+1)*numLista
                if len(misFinales)<maximo:
                    maximo=len(misFinales)
                finales=misFinales[r*numLista:maximo]
                if verbose:
                    print(r,"Finales",r*numLista,maximo)
                r+=1
                

                finales=list(finales)


                cpus=mp.cpu_count()
                cpus=32
                k=0
                argus=[]
                num=int(np.floor(len(finales)/(cpus)))
                if num==0:
                    num=1
                while k*num<len(finales):
                    argus.append((k*num,(k+1)*num))
                    k+=1
                    
                    
                print("antes de with")
                with mp.Pool(processes=cpus) as mp_pool:
                    results=mp_pool.starmap(chequea,argus)

                print("antes de for")
                for result in results:
                    lista.update(result)
                



            if i==tope:
                print("Fase2",time.time()-st2)
            st3=time.time()
            
            cont2=1
            lista2=set()
            listaL2=list(lista)
            
            
            cpus=mp.cpu_count()
            cpus=32

            k=0
            argus=[]
            num=int(np.floor(len(listaL2)/(cpus)))
            if num==0:
                num=1
            while k*num<len(listaL2):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analizaSalto,argus)

            for result in results:
                lista2.update(result[0])
                misSoportes.update(result[1])
                for sop in result[1]:
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                cont2+=1
            
                
                
            if verbose:
                print()
                print("Lista2",len(lista2))
                print()
            
            cutsetsT=set()
            
            if i==tope:
                print("Fase3",time.time()-st3)
                miLista2=lista2
            st4=time.time()
            
            if verbose:
                print("Sin salto")
            descartados=set()
            cont=0
            cpus=32
            miLista2=list(lista2)
            listaT=set()

            k=0
            argus=[]
            num=int(np.floor(len(miLista2)/(cpus)))
            if num==0:
                num=1
            while k*num<len(miLista2):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analiza,argus)

            for result in results:
                listaT.update(result[0])
                misSoportes.update(result[1])
                for sop in result[1]:
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
            
            cutsets.append(set(listaT))
            if i==tope:
                print("Fase4",time.time()-st4)
            todosMCS.update(listaT)   
        if i>1:
            tiempos.append([st1,st2,st3])
    
    return(cutsets)
                        

In [12]:
model_list=[nombre+"-Aerobic-rephosphorylation-of-ATP-from-glucose.xml", nombre+"-Aerobic-rephosphorylation-of-GTP.xml",
               nombre+"-Aerobic-rephosphorylation-of-CTP.xml", nombre+"-Aerobic-rephosphorylation-of-UTP.xml",
               
              nombre+"-ATP-de-novo-synthesis.xml",nombre+"-CTP-de-novo-synthesis.xml",nombre+"-GTP-de-novo-synthesis.xml",
               
               nombre+"-UTP-de-novo-synthesis.xml",nombre+"-dATP-de-novo-synthesis.xml",nombre+"-dCTP-de-novo-synthesis.xml",
               
               nombre+"-dGTP-de-novo-synthesis.xml", nombre+"-dTTP-de-novo-synthesis.xml",nombre+"-Histidine-uptake.xml",
               
               nombre+"-Isoleucine-uptake.xml",nombre+"-Leucine-uptake.xml",nombre+"-Lysine-uptake.xml", nombre+"-Methionine-uptake.xml",
               
               nombre+"-Phenyalanine-uptake.xml",nombre+"-Threonine-uptake.xml",nombre+"-TryptophanUptake.xml",nombre+"-Valine-uptake.xml",
               
               nombre+"-G3P-de-novo-synthesis.xml", nombre+"-Mitochondrial-AcCoA-synthesis.xml",nombre+"-Mitochondrial-AKG-synthesis.xml",
               
               nombre+"-Erythrose4P-de-novo.xml",nombre+"-Fructose6P-de-novo.xml",nombre+"-GAP-de-novo.xml",nombre+"-Glucose6P-de-novo.xml",
               
               nombre+"-Mitochondrial-OAA-de-novo.xml", nombre+"-PEP-de-novo.xml", nombre+"-Pyruvate-de-novo.xml",nombre+"-Ribose5P-de-novo.xml",
               
               nombre+"-Mitochondrial-SuccCoA-synthesis.xml",nombre+"-Cholesterol-synthesis.xml", nombre+"-Protein-synthesis.xml", 
               
               nombre+"-Oxidative-phosphorilation.xml", nombre+"-Oxidative-decarboxilation.xml",nombre+"-Krebs-cycle-NADH.xml",
               
               nombre+"-Ubiquinol-to-proton.xml", nombre+"-Ubiquinol-to-ATP.xml", nombre+"-BOxidationSatFA.xml",
               
               nombre+"-Beta-oxidation-of-long-chain-FA.xml", nombre+"-BOxidationOddFA.xml", nombre+"-BOxidationUnSatFAn-9.xml",
               
               nombre+"-BOxidationUnSatFAn-6.xml",  nombre+"BOxidationNEFAs.xml" ,nombre+"-CholineUptake.xml",nombre+"-InositolUptake.xml",
               
               nombre+"-Phosphatidylcholine-de-novo-synthesis.xml",nombre+"-Phosphatidylethanolamine-de-novo-synthesis.xml", 
               
               nombre+"-Phosphatidylserine-de-novo-synthesis.xml",nombre+"-Phosphatidylinositol-de-novo-synthesis.xml",
                
               nombre+"-Thiamin-phosphorylation-to-TPP.xml", nombre+"-Coenzyme-A-synthesis-from-pantothenate.xml",
               
               nombre+"-FAD-synthesis-from-riboflavin.xml",nombre+"-Heme-biosynthesis.xml", nombre+"-BiomassProduction.xml"]
'''

Apply single_gene_deletion() to the model list

In [ ]:

for i in model_list:
    print(i)
    model=cobra.io.read_sbml_model(i)
    
    dr=single_gene_deletion(model)
    nombre="MT"+str(1+model_list.index(i))+"-SGD-"+nombre
    with open(nombre, "wb") as f:
        pickle.dump(dr, f)


Apply SL program to the model list

In [ ]:
for j in model_list:
    print(j)
    model=cobra.io.read_sbml_model(j)
        
    misReacciones, misValores = [],[[]]
    modelo=model.copy() # Sin metabolic task
    listaGenes=list(modelo.genes)
    misSoportes=set()
    sopG=dict()
    tope=2
    asociados,reacciones=asociar(modelo,tope)

    tiempos=[]
    powers=dict()
    CS2=CSV2(modelo,tope,misReacciones,misValores,True,set())    

    
    for i in range(10):
        with modelo:
            modelo.objective=np.random.choice(list(modelo.reactions),1)[0]
            sol=modelo.optimize()
            sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol.fluxes[j])>10**-8])
            sopG[sop]=soporteG(sop,tope,modelo)
            misSoportes.add(sop)
        
    cutsets=set()
    for c in range(len(CS2)):
        print(c,len(CS2[c]))
    for caso in CS2[c]:
        cutsets.add(caso)
        
    nombre="MT"+str(1+model_list.index(j))+"-CS-"+nombre
    with open(nombre, "wb") as f:
        pickle.dump(cutsets, f)